In [ ]:
import pandas as pd
df_updated = pd.read_csv("BLIP_results_mit_states.csv")

In [ ]:
df_correct=df_updated[df_updated["correct_answer"] == df_updated["generated_answer"]]
df_correct

In [ ]:
df = df_correct
color_df = df[df['state'] == 'color']
shape_df = df[df['state'] == 'shape']
texture_df = df[df['state'] == 'texture']

def sample_state(state_df):
    correct_first = state_df[state_df['clean_prompt'].str.contains(f' {state_df["correct_answer"].iloc[0]} or')]
    correct_second = state_df[state_df['clean_prompt'].str.contains(f'or {state_df["correct_answer"].iloc[0]}')]

    # Sample to ensure at least one of each correct answer
    unique_correct_answers = state_df['correct_answer'].unique()
    samples = []
    
    for answer in unique_correct_answers:
        correct_first_sample = correct_first[correct_first['correct_answer'] == answer]
        correct_second_sample = correct_second[correct_second['correct_answer'] == answer]
        
        if not correct_first_sample.empty:
            samples.append(correct_first_sample.sample(1))
        if not correct_second_sample.empty:
            samples.append(correct_second_sample.sample(1))
    
    correct_first_remaining = correct_first.drop(pd.concat(samples).index, errors='ignore')
    correct_second_remaining = correct_second.drop(pd.concat(samples).index, errors='ignore')

    correct_first_samples = pd.concat([pd.concat(samples), correct_first_remaining.sample(250 - len(samples)//2, replace=True)])
    correct_second_samples = pd.concat([pd.concat(samples), correct_second_remaining.sample(250 - len(samples)//2, replace=True)])

    combined_samples = pd.concat([correct_first_samples, correct_second_samples]).sample(500, replace=True)

    return combined_samples

# Sample 500 rows from each state
color_samples = sample_state(df[df['state'] == 'color'])
shape_samples = sample_state(df[df['state'] == 'shape'])
texture_samples = sample_state(df[df['state'] == 'texture'])

# Combine all samples into one DataFrame
final_samples = pd.concat([color_samples, shape_samples, texture_samples])

# Display the final sampled DataFrame
final_samples

In [ ]:
final_samples = final_samples.reset_index(drop=True)

In [ ]:
final_samples['correct_first'] = final_samples.apply(lambda row: row['clean_prompt'].split(' ')[3] == row['correct_answer'], axis=1)

final_samples

In [ ]:
final_samples.to_csv("BLIP_final_mit_states.csv", index=False)